In [1]:
# pip install -r requirements.txt

In [2]:
# Imports
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import ta
from datetime import date, timedelta

# Dynamic timeline (end is exclusive, so add a day to grab latest close)
start_date = date(2015, 1, 1)
end_date = date.today() + timedelta(days=1)


In [3]:
# Cross-sector mix for future modeling
tickers = ["AAPL","MSFT","AMZN","GOOGL","NVDA","TSLA","JPM","WMT",
           "DAL","UAL","LMT","RTX","NOC","XOM"]

print(f"{len(tickers)} tickers")

14 tickers


In [4]:
# Download price data
data = yf.download(tickers, start=start_date, end=end_date, auto_adjust=True)

[*********************100%***********************]  14 of 14 completed


In [5]:
# Flatten multi-index into one row per ticker and date
data_flat = data.stack(level=1, future_stack=True).reset_index()

data_flat.rename(columns={
    "level_1": "Ticker",
    "Adj Close": "AdjClose",
    "Close": "Close",
    "Open": "Open",
    "High": "High",
    "Low": "Low",
    "Volume": "Volume"
}, inplace=True)

data_flat.head()

Price,Date,Ticker,Close,High,Low,Open,Volume
0,2015-01-02,AAPL,24.237553,24.705322,23.798602,24.694237,212818400
1,2015-01-02,AMZN,15.426000,15.737500,15.348000,15.629000,55664000
2,2015-01-02,DAL,43.064987,43.791784,42.653426,43.712974,8637300
3,2015-01-02,GOOGL,26.278946,26.589102,26.196070,26.430301,26480000
4,2015-01-02,JPM,46.720932,47.072328,46.406916,46.489158,12600000


In [6]:
data_flat["Return"] = data_flat.groupby("Ticker")["Close"].pct_change()

data_flat["RollingVol"] = (
    data_flat.groupby("Ticker")["Return"]
    .rolling(window=10)
    .std()
    .reset_index(0, drop=True)
)

data_flat["RSI"] = data_flat.groupby("Ticker")["Close"].transform(
    lambda x: ta.momentum.rsi(x, window=14)
)

# SMA ratios instead of absolute levels to avoid price scale leakage
data_flat["SMA_20"] = data_flat.groupby("Ticker")["Close"].transform(lambda x: x.rolling(20).mean())
data_flat["SMA_50"] = data_flat.groupby("Ticker")["Close"].transform(lambda x: x.rolling(50).mean())

data_flat["Price_to_SMA20"] = data_flat["Close"] / data_flat["SMA_20"]
data_flat["Price_to_SMA50"] = data_flat["Close"] / data_flat["SMA_50"]
data_flat["SMA20_to_SMA50"] = data_flat["SMA_20"] / data_flat["SMA_50"]

data_flat.drop(columns=["SMA_20", "SMA_50"], inplace=True)

data_flat["Volume_Z"] = (
    data_flat.groupby("Ticker")["Volume"]
    .transform(lambda x: (x - x.rolling(20).mean()) / x.rolling(20).std())
)

In [7]:
# Download VIX index using synchronized dynamic dates
macro = yf.download(["^VIX"], start=start_date, end=end_date, auto_adjust=True)

macro.columns = [col[0] for col in macro.columns]
macro = macro.reset_index().rename(columns={"Close":"VIX"})

data_flat = data_flat.merge(macro[["Date","VIX"]], on="Date", how="left")

[*********************100%***********************]  1 of 1 completed


In [8]:
data_flat = data_flat.sort_values(['Ticker', 'Date']).reset_index(drop=True)

# Lagged_Return captures current day's momentum as a feature for t+1 prediction
data_flat["Lagged_Return"] = data_flat["Return"]

# Shift labels forward by 1 period to align features at t with outcomes at t+1 (handles look-ahead bias)
data_flat["Next_Day_Return"] = data_flat.groupby("Ticker")["Return"].shift(-1)

# Preserve NaN for terminal rows
data_flat["Direction"] = np.where(
    data_flat["Next_Day_Return"].isna(),
    np.nan,
    (data_flat["Next_Day_Return"] > 0).astype(float)
)

# VolSpike uses expanding quantile with 126-day minimum (6 months) to establish stable baseline and reduce cold-start noise
next_day_vol = data_flat.groupby("Ticker")["RollingVol"].shift(-1)
expanding_threshold = data_flat.groupby("Ticker")["RollingVol"].transform(
    lambda x: x.expanding(min_periods=126).quantile(0.8)
)

data_flat["VolSpike"] = np.where(
    next_day_vol.isna() | expanding_threshold.isna(),
    np.nan,
    (next_day_vol > expanding_threshold).astype(float)
)

data_flat.drop(columns=["Next_Day_Return"], inplace=True)

print(f"NaN Direction: {data_flat['Direction'].isna().sum()}")
print(f"NaN VolSpike: {data_flat['VolSpike'].isna().sum()}")

NaN Direction: 14
NaN VolSpike: 1904


In [9]:
from pathlib import Path
Path('data').mkdir(exist_ok=True)

In [10]:
# Save FULL dataset including incomplete features for EDA price history, Jan 2015 - Present
data_flat.to_csv("data/merged_features_full.csv", index=False)

# Drop rows after all feature/label engineering to remove cold-start and terminal rows
data_flat_clean = data_flat.dropna().copy()

data_flat_clean["Direction"] = data_flat_clean["Direction"].astype(int)
data_flat_clean["VolSpike"] = data_flat_clean["VolSpike"].astype(int)

# Save CLEAN dataset (model-ready) for feature analysis and modeling, July 2015 - Present
data_flat_clean.to_csv("data/merged_features_clean.csv", index=False)

print(f"Full dataset (with incomplete features): {data_flat.shape[0]} rows")
print(f"Clean dataset (model-ready): {data_flat_clean.shape[0]} rows")
print(f"Rows dropped (incomplete features): {data_flat.shape[0] - data_flat_clean.shape[0]}")
print(f"\nClean dataset stats:")
print(f"Direction: {data_flat_clean['Direction'].value_counts().to_dict()}")
print(f"VolSpike: {data_flat_clean['VolSpike'].value_counts().to_dict()}")

data_flat_clean.head(10)

Full dataset (with incomplete features): 38682 rows
Clean dataset (model-ready): 36778 rows
Rows dropped (incomplete features): 1904

Clean dataset stats:
Direction: {1: 19424, 0: 17354}
VolSpike: {0: 27500, 1: 9278}


,Date,Ticker,Close,High,Low,Open,Volume,Return,RollingVol,RSI,Price_to_SMA20,Price_to_SMA50,SMA20_to_SMA50,Volume_Z,VIX,Lagged_Return,Direction,VolSpike
135,2015-07-17,AAPL,28.969582,28.969582,28.676802,28.848895,184658800,0.008637,0.016381,59.275631,1.028554,1.014973,0.986796,0.209108,11.95,0.008637,1,0
136,2015-07-20,AAPL,29.517155,29.718300,29.210962,29.271307,235600800,0.018902,0.016975,64.539780,1.045726,1.033056,0.987884,1.110108,12.25,0.018902,0,1
137,2015-07-21,AAPL,29.222134,29.707120,29.126032,29.691477,307025600,-0.009995,0.017494,60.036926,1.033989,1.022230,0.988628,2.005310,12.22,-0.009995,0,1
138,2015-07-22,AAPL,27.986206,28.048785,27.264312,27.264312,461802400,-0.042294,0.021186,45.663651,0.990966,0.979164,0.988090,3.038030,12.12,-0.042294,0,1
139,2015-07-23,AAPL,27.972795,28.404142,27.950444,28.205230,203998000,-0.000479,0.019694,45.536262,0.991649,0.978804,0.987046,0.060208,12.64,-0.000479,0,1
140,2015-07-24,AAPL,27.825283,28.102418,27.691186,28.008550,168649200,-0.005273,0.018193,44.079574,0.987593,0.973872,0.986106,-0.379597,13.74,-0.005273,0,0
141,2015-07-27,AAPL,27.438635,27.626373,27.293364,27.510154,177822000,-0.013896,0.017526,40.428901,0.975410,0.961269,0.985502,-0.272475,15.60,-0.013896,1,0
142,2015-07-28,AAPL,27.574972,27.693427,27.389472,27.574972,134472400,0.004969,0.017669,42.245280,0.980705,0.966861,0.985884,-0.735659,13.44,0.004969,0,0
143,2015-07-29,AAPL,27.487806,27.601790,27.326889,27.523567,148046800,-0.003161,0.017219,41.376635,0.978554,0.964894,0.986041,-0.556962,12.50,-0.003161,0,0
144,2015-07-30,AAPL,27.349245,27.393944,27.201737,27.338070,134513200,-0.005041,0.016244,39.969696,0.975262,0.961191,0.985572,-0.725579,12.13,-0.005041,0,0


### Dataset Outputs

Two datasets are saved for different purposes:

1. **`merged_features_full.csv`** - Complete dataset including rows with incomplete features
   - **Used for**: EDA price history, understanding full market timeline
   - **Contains**: All rows from Jan 2015 onwards, including early 2015 with NaN indicators
   - **Rows**: ~38,668

2. **`merged_features_clean.csv`** - Model-ready dataset with complete features only
   - **Use for**: Feature analysis, correlation matrices, modeling, and predictions
   - **Contains**: Only rows where all technical indicators have complete lookback windows
   - **Rows**: ~36,764
   - **Start date**: July 17, 2015, after 50-day MA, 126-day expanding window requirements are satisfied

The separation created here ensures that our EDA can show the complete market picture while modeling uses only reliable, fully-calculated features.